In [1]:
'''
        Importing Library
'''
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
print("Importing module...")
import torch
import numpy as np
import torch

from PIL import Image
from src.utils.ODBTC import ODBTC
from src.utils.Codec import Codec
print("Importing module sucess")

Importing module...
Importing module sucess


In [2]:
img_arr = np.load('../dataset/npy/dataset_npy.npy')
print(img_arr.shape)
img_target = img_arr[0]

(400, 1, 224, 224)


In [3]:
img_target = img_target[0]
print(img_target.shape)

(224, 224)


In [18]:
codec = Codec(4,4,'bilinear')

In [19]:
new_tensor = torch.Tensor(img_target)
print(new_tensor.shape)
img_temp = codec.btc(img_target)

torch.Size([224, 224])


In [21]:
data = Image.fromarray(img_temp.numpy().astype(np.uint8))
data.save('../dataset/temp/len_bmp.png')